In [3]:
# iPyTest allows us to solve AoC using test-driven development principals.
import ipytest
ipytest.autoconfig(addopts=['--color=no'])

In [8]:
# Modules to support development
import os
import re
import collections
import itertools
import functools
import logging
import pprint
import numpy as np
import heapq

In [89]:
def read_input(puzzle_input, part2=False):
    with open(puzzle_input) as f:
        dd = [ xx.strip() for xx in f.readlines() ]

    # parse
    x_dim = [None, None]
    y_dim = [None, None]
    paths = []
    for ll in dd:
        parts = ll.split(" -> ")
        path = []
        for part in parts:
            xx, yy = part.split(",")
            xx, yy = int(xx), int(yy)

            if x_dim[0] is None or xx < x_dim[0]:
                x_dim[0] = xx
            if x_dim[1] is None or xx > x_dim[1]:
                x_dim[1] = xx
            if y_dim[0] is None or yy < y_dim[0]:
                y_dim[0] = yy
            if y_dim[1] is None or yy > y_dim[1]:
                y_dim[1] = yy

            path.append((xx,yy))
        paths.append(path)
                  
    assert x_dim[0] >= 0
    assert y_dim[0] >= 0

    x_size = x_dim[1] - x_dim[0] + 3
    y_size = y_dim[1] + 2

    cave = np.ndarray(shape=(y_size, x_size), dtype='int8')
    cave.fill(ord('.'))
    for path in paths:
        for ii, part in enumerate(path):
            if ii == 0:
                continue
            xx_a, yy_a = part
            xx_b, yy_b = path[ii-1]

            xx_a -= (x_dim[0] - 1)
            xx_b -= (x_dim[0] - 1)

            xx_s = min(xx_a, xx_b)
            yy_s = min(yy_a, yy_b)
            xx_e = max(xx_a, xx_b)
            yy_e = max(yy_a, yy_b)
            
            xx, yy = xx_s, yy_s
            while yy <= yy_e:
                xx = xx_s
                while xx <= xx_e:
                    cave[(yy, xx)] = ord('#')
                    xx += 1
                yy += 1

    return cave, x_dim[0] - 1, 0

def test_read_input():
    cave, _ , _ = read_input(os.path.join(os.path.join("..", "dat", "day14_test.txt")))

    for ii, row in enumerate(cave):
        print(ii, end=" ")
        for col in row:
            print(chr(col), end="")
        print("")
    #display(cave)

test_read_input()

0 ............
1 ............
2 ............
3 ............
4 .....#...##.
5 .....#...#..
6 ...###...#..
7 .........#..
8 .........#..
9 .#########..
10 ............


In [88]:


def drop_sand(cave, x_off, y_off, start=(500,0)):
    pos = start
    
    while True:
        next_pos = (pos[0], pos[1] + 1)
        if next_pos[1] >= cave.shape[0]:
            cave[pos[1], pos[0]-x_off] = ord('~')
            return True
        
        if cave[next_pos[1], next_pos[0]-x_off] == ord('.'):
            pos = next_pos
            continue

        next_pos = (pos[0]-1, pos[1] + 1)
        if cave[next_pos[1], next_pos[0]-x_off] == ord('.'):
            pos = next_pos
            continue
    
        next_pos = (pos[0]+1, pos[1] + 1)
        if cave[next_pos[1], next_pos[0]-x_off] == ord('.'):
            pos = next_pos
            continue

        cave[pos[1], pos[0]-x_off] = ord('o')
        break

    return False

cave, x_off, y_off = read_input(os.path.join(os.path.join("..", "dat", "day14.txt")))

complete = False
dropped = 0
while complete == False:
    complete = drop_sand(cave, x_off, y_off)
    if not complete:
        dropped += 1
    
print("Dropped %s grains" % dropped)
for ii, row in enumerate(cave):
    print("%02d" % ii, end=" ")
    for col in row:
        print(chr(col), end="")
    print("")

Dropped 592 grains
00 ...........................................................
01 ...........................................................
02 ...........................................................
03 ...........................................................
04 ...........................................................
05 ...........................................................
06 ...........................................................
07 ...........................................................
08 ...........................................................
09 ...........................................................
10 ...........................................................
11 ...........................................................
12 ...........................................................
13 ......................................o....................
14 .....................................ooo...................
15 .................................

In [98]:
def read_input(puzzle_input, part2=False):
    with open(puzzle_input) as f:
        dd = [ xx.strip() for xx in f.readlines() ]

    # parse
    x_dim = [None, None]
    y_dim = [None, None]
    paths = []
    for ll in dd:
        parts = ll.split(" -> ")
        path = []
        for part in parts:
            xx, yy = part.split(",")
            xx, yy = int(xx), int(yy)

            if x_dim[0] is None or xx < x_dim[0]:
                x_dim[0] = xx
            if x_dim[1] is None or xx > x_dim[1]:
                x_dim[1] = xx
            if y_dim[0] is None or yy < y_dim[0]:
                y_dim[0] = yy
            if y_dim[1] is None or yy > y_dim[1]:
                y_dim[1] = yy

            path.append((xx,yy))
        paths.append(path)
                  
    cave = {}
    for path in paths:
        for ii, part in enumerate(path):
            if ii == 0:
                continue
            xx_a, yy_a = part
            xx_b, yy_b = path[ii-1]

            xx_s = min(xx_a, xx_b)
            yy_s = min(yy_a, yy_b)
            xx_e = max(xx_a, xx_b)
            yy_e = max(yy_a, yy_b)
            
            xx, yy = xx_s, yy_s
            while yy <= yy_e:
                xx = xx_s
                while xx <= xx_e:
                    cave.setdefault(yy, {})[xx] = '#'
                    xx += 1
                yy += 1

    return cave

def print_cave(cave):
    yy_s = min(cave.keys())
    yy_e = max(cave.keys())

    xx_s = min(cave[yy_s].keys())
    xx_e = max(cave[yy_s].keys())

    for yy, row in cave.items():
        xx_s = min(xx_s, min(row.keys()))
        xx_e = max(xx_e, max(row.keys()))


    for yy in range(yy_s, yy_e+1):
        print("%02d" % yy, end=" ")
        for xx in range(xx_s, xx_e+1):
            print(cave.get(yy, {}).get(xx, '.'), end="")
        print("")

def test_read_input():
    cave = read_input(os.path.join(os.path.join("..", "dat", "day14_test.txt")))
    print_cave(cave)

test_read_input()

04 ....#...##
05 ....#...#.
06 ..###...#.
07 ........#.
08 ........#.
09 #########.


In [106]:


def drop_sand(cave, floor, start=(500,0)):
    pos = start
    
    while True:
        next_pos = (pos[0], pos[1] + 1)

        # We are resting on the floor
        if next_pos[1] == floor:
            cave.setdefault(pos[1], {})[pos[0]] = 'o'
            break

        # Check if we can fall
        if cave.get(next_pos[1], {}).get(next_pos[0]) == None:
            pos = next_pos
            continue

        next_pos = (pos[0]-1, pos[1] + 1)
        if cave.get(next_pos[1], {}).get(next_pos[0]) == None:
            pos = next_pos
            continue
    
        next_pos = (pos[0]+1, pos[1] + 1)
        if cave.get(next_pos[1], {}).get(next_pos[0]) == None:
            pos = next_pos
            continue

        # We are stuck
        if pos == start:
            return True
        
        cave.setdefault(pos[1], {})[pos[0]] = 'o'
        break

    return False

cave = read_input(os.path.join(os.path.join("..", "dat", "day14.txt")))
floor = max(cave.keys()) + 2

complete = False
dropped = 0
while complete == False:
    complete = drop_sand(cave, floor)
    dropped += 1
print("Dropped %s grains" % dropped)
print_cave(cave)

Dropped 30367 grains
01 ................................................................................................................................................................................ooo................................................................................................................................................................................
02 ...............................................................................................................................................................................ooooo...............................................................................................................................................................................
03 ..............................................................................................................................................................................ooooooo.............................................................................